# Assignment 1 
**Group B** \
Students: Elodie-Angélique Lendi, Mridhula Jayasankar, Clarice Jasmine Mihalache



In [47]:
# Imports cell
import pandas as pd
import sympy as sp

## Problem 1

In [ ]:
df = pd.read_csv("periodic_table.csv")
print(df.head()) #just for now to see if it can be loaded for everyone

   AtomicNumber Symbol       Name  AtomicMass CPKHexColor  \
0             1      H   Hydrogen    1.008000      FFFFFF   
1             2     He     Helium    4.002600      D9FFFF   
2             3     Li    Lithium    7.000000      CC80FF   
3             4     Be  Beryllium    9.012183      C2FF00   
4             5      B      Boron   10.810000      FFB5B5   

  ElectronConfiguration  Electronegativity  AtomicRadius  IonizationEnergy  \
0                   1s1               2.20         120.0            13.598   
1                   1s2                NaN         140.0            24.587   
2               [He]2s1               0.98         182.0             5.392   
3               [He]2s2               1.57         153.0             9.323   
4           [He]2s2 2p1               2.04         192.0             8.298   

   ElectronAffinity OxidationStates StandardState  MeltingPoint  BoilingPoint  \
0             0.754          +1, -1           Gas         13.81         20.28   
1 

## Problem 2

In [ ]:
def individual_element(mol):
    elements = {}
    i = 0
    while i < len(mol):
        if mol[i].isupper():
            el = mol[i]
            i += 1
            if i < len(mol) and mol[i].islower():
                el += mol[i]
                i += 1
            num = ""
            while i < len(mol) and mol[i].isdigit():
                num += mol[i]
                i += 1
            count = int(num) if num else 1
            elements[el] = elements.get(el, 0) + count
        else:
            i += 1
    return elements

In this function "individual_elements" we decompose all our reactants in order to know which atoms are present in our reaction and to know how many of each atom are present in each molecule. This is very usuful in order to build our matrix later on. 

As all the elements start with a capital letter each element was searched in this way. In order to avoid the problem for ("Na") being divided into ("N", "a"), this function also looks at the letter following the capitalized letter to see if there is a lowercase character following it. Now our variable $el$ contains the name of our element. In order to get an idea of the number of atom present, this function also looks at the numbers following the element using the function ".isdigits". If there is no number following the element, a default value of 1 is taken. 

Finally the elements an the element count is added into the dictionnary. 

In [72]:
def balance_reaction(reactants, products):
    # Step 1
    all_elements = set()
    for mol in reactants + products:
        all_elements.update(individual_element(mol).keys())
    all_elements = sorted(all_elements)

    # Step 2
    species = reactants + products
    matrix = []
    for el in all_elements:
        row = []
        for i, mol in enumerate(species):
            counts = individual_element(mol)
            cnt = counts.get(el, 0)
            row.append(cnt if i < len(reactants) else -cnt)
        matrix.append(row)

    A = sp.Matrix(matrix)

    # Step 3
    nullspace = A.nullspace()
    if not nullspace:
        raise ValueError("No solution found for this reaction.")
    
    vec = nullspace[0] 
    denoms = [r.q for r in vec]
    lcm_den = sp.lcm(denoms)
    int_vec = [int(r * lcm_den) for r in vec]

    from math import gcd
    from functools import reduce
    g = reduce(gcd, [abs(x) for x in int_vec if x != 0])
    int_vec = [x // g for x in int_vec]

    return int_vec


The main part of the code can be divided in three subparts each with a particular aim. 

As said earlier a matrix will be used in order to equilibrate the reaction. Hence we need to know whcih elements are present in the reaction without taking duplicates. Using Sets in the first part allows to do that. 

In the second part we look at both the reactants and the products. Taking the set that was created above, we can compare for each element the number of times it appears in each molecule. In this part, we take a positive value for the reactants and a negative value for the product, this is due to the atom conservation law . It is in this part that the matrix is created, where the row represent the individual elements and the columns represent the molecules (the reactants first, and the products after). 

In this final part, we find the solutions to equilibrate the reaction and find the correct stoechiometry. The function ".nullspace()" returns a list of basis vector, where each vector in the nullspace is a possible answer for the ratios of coefficients. However, this method could give us ratios instead of integer values for the stoechiometry. Hence we find the smallest common multiple. 

Once all these steps have been executed, the final answers shows up. 

In [74]:
print(balance_reaction(reactants=["H2", "O2"], products=["H2O"]))
print(balance_reaction(reactants=["N2", "H2"], products=["NH3"]))
print(balance_reaction(reactants=["SO2", "O2"], products=["SO3"])) 

[2, 1, 2]
[1, 3, 2]
[2, 1, 2]


## Problem 3